In [9]:
!git clone https://huggingface.co/spaces/lambdaofgod/paperswithcode_nbow /tmp/nbow

Cloning into '/tmp/nbow'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 83 (delta 32), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (83/83), 18.20 KiB | 1.52 MiB/s, done.


In [10]:
import datasets
import sys

sys.path.append("/tmp/nbow")
import search_utils

In [13]:
def get_model_paths(model_name):
    query_encoder_name = "lambdaofgod/query-" + model_name
    document_encoder_name = "lambdaofgod/document-" + model_name
    return query_encoder_name, document_encoder_name

## Model names from huggingface hub

In [15]:
model_names = [
    'dependencies-nbow-nbow-mnrl',
    'readme-nbow-nbow-mnrl',
    'titles-nbow-nbow-mnrl',
    'titles_dependencies-nbow-nbow-mnrl',
    'readme_dependencies-nbow-nbow-mnrl'
]

### Select model name

In [17]:
model_name = model_names[0]

### Prepare retrieval df

The dataframe will use columns inferred from model name

In [6]:
doc_cols = search_utils.get_doc_cols(model_names[0])
retrieval_df = search_utils.merge_cols(
    search_utils.get_retrieval_df(text_list_cols=["titles"]), doc_cols
)

2023-01-06 15:01:36.079 WARNING datasets.builder: Using custom data configuration lambdaofgod___pwc_repositories_with_dependencies-b5c0ffee4502e15e
2023-01-06 15:01:36.081 WARNING datasets.builder: Reusing dataset parquet (/home/kuba/.cache/huggingface/datasets/parquet/lambdaofgod___pwc_repositories_with_dependencies-b5c0ffee4502e15e/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121)


  0%|          | 0/1 [00:00<?, ?it/s]

### Setup retrieval_pipeline

In [7]:
retrieval_pipeline = search_utils.RetrievalPipelineWrapper.setup_from_encoder_names(
    *get_model_paths(model_name),
    retrieval_df["document"],
    retrieval_df,
    "cuda"
)

2023-01-06 15:01:36.160 INFO    sentence_transformers.SentenceTransformer: Load pretrained SentenceTransformer: lambdaofgod/document-dependencies-nbow-nbow-mnrl
2023-01-06 15:01:36.254 INFO    gensim.summarization.textcleaner: 'pattern' package not found; tag filters are not available for English
2023-01-06 15:01:36.382 INFO    sentence_transformers.models.WordWeights: 0 of 53559 words without a weighting value. Set weight to 1
2023-01-06 15:01:36.384 INFO    sentence_transformers.SentenceTransformer: Load pretrained SentenceTransformer: lambdaofgod/query-dependencies-nbow-nbow-mnrl
2023-01-06 15:01:36.393 INFO    sentence_transformers.models.WordWeights: 0 of 4395 words without a weighting value. Set weight to 1


Batches:   0%|          | 0/137 [00:00<?, ?it/s]

2023-01-06 15:01:44.285 INFO    nmslib: M                   = 16
2023-01-06 15:01:44.285 INFO    nmslib: indexThreadQty      = 32
2023-01-06 15:01:44.286 INFO    nmslib: efConstruction      = 200
2023-01-06 15:01:44.286 INFO    nmslib: maxM			          = 16
2023-01-06 15:01:44.286 INFO    nmslib: maxM0			          = 32
2023-01-06 15:01:44.286 INFO    nmslib: mult                = 0.360674
2023-01-06 15:01:44.287 INFO    nmslib: skip_optimized_index= 0
2023-01-06 15:01:44.287 INFO    nmslib: delaunay_type       = 2
2023-01-06 15:01:44.287 INFO    nmslib: Set HNSW query-time parameters:
2023-01-06 15:01:44.288 INFO    nmslib: ef(Search)         =20
2023-01-06 15:01:44.288 INFO    nmslib: algoType           =2
2023-01-06 15:01:44.329 INFO    nmslib: 
The vector space is CosineSimilarity
2023-01-06 15:01:44.329 INFO    nmslib: Vector length=200
2023-01-06 15:01:44.330 INFO    nmslib: searchMethod			  = 3
2023-01-06 15:01:44.330 INFO    nmslib: Making optimized index
2023-01-06 15:01:44.332

# Search

In [18]:
query = "metric learning"

In [21]:
retrieval_pipeline.search(query, 10, description_length=10, additional_shown_cols=doc_cols)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,repo,tasks,link,distance,dependencies
0,DagnyT/hardnet,"['image retrieval', 'patch matching']",https://github.com/DagnyT/hardnet,0.524190,code/Losses.py code/download_all_datasets.py e...
1,Phoebe-star/part_aligned,['person re identification'],https://github.com/Phoebe-star/part_aligned,0.545965,heads/fc1024_normalize.py cmc.py heads/fc1024....
2,Phoebe-star/AlignedReID,['person re identification'],https://github.com/Phoebe-star/AlignedReID,0.550748,cmc.py heads/fc1024.py big_dataset_label.py ne...
3,vinnik-dmitry07/PlaceRecognition,"['visual place recognition', 'image retrieval']",https://github.com/vinnik-dmitry07/PlaceRecogn...,0.556338,main.py hard_triplet_loss.py netvlad.py _pairw...
4,MayraMacasC/AnomalyDetection,['anomaly detection'],https://github.com/MayraMacasC/AnomalyDetection,0.566667,Anomaly Detection/S_Distance_Matrix.py STAE-AD...
5,kridgeway/f-statistic-loss-nips-2018,['few shot learning'],https://github.com/kridgeway/f-statistic-loss-...,0.567227,models/deep_metric_learning.py data/sprites.py...
6,Parth27/ActionRecognitionVideos,['action recognition'],https://github.com/Parth27/ActionRecognitionVi...,0.569983,Code/Process_dataset.py Code/GSM.py Code/Cosin...
7,empty16/hardnet.pytorch,"['image retrieval', 'patch matching']",https://github.com/empty16/hardnet.pytorch,0.582309,examples/extract_hardnet_desc_from_hpatches_fi...
8,crazydemo/Progressive-Multi-stage-Feature-Mix-...,['person re identification'],https://github.com/crazydemo/Progressive-Multi...,0.595354,models/resnet.py utils/validation_metrics.py d...
9,i-samenko/Triplet-net,['word embeddings'],https://github.com/i-samenko/Triplet-net,0.596403,trainer.py dataset.py models.py utils.py quali...
